<a href="https://colab.research.google.com/github/chi777/wordle/blob/main/Wordle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# Exact calculation of expected number of guesses given even distribution
# of solution words

In [19]:
from collections import Counter
import sys
import logging
from math import log
import pandas as pd
from functools import lru_cache
from os.path import exists
import pickle

In [20]:
# Get guess and solution word list
dir = '/content/drive/MyDrive/ColabData/Wordle/'
with open(dir + 'wordlist_solutions.txt') as word_file:
  solutions = word_file.read().split()
with open(dir + 'wordlist_guesses.txt') as word_file:
  guesses = word_file.read().split()

In [21]:
# Calculate entropy for distribution
def distribution_to_entropy(distribution):
  """
  Takes distribution as list and returns entropy value
  """
  total = sum(distribution)
  ent = [log(d,2) * d / total for d in distribution]
  return(sum(ent))

dist = [1, 1, 1, 2]
print(distribution_to_entropy(dist))

0.4


In [22]:
def calc_feedback(true_word, test_word):
  """
  Get feedback string for true_word given test_word
  """
  # Set up feedback list
  fbl = [None] * 5
  # Set up counter for true_word
  count = Counter(true_word)
  # Check full matches
  for i in range(5):
    if true_word[i] == test_word[i]:
      fbl[i] = 'r'
      count[true_word[i]] -= 1
  # Check partial matches
  for i in range(5):
    if fbl[i] != None:
      continue
    c = test_word[i]
    if count[c] > 0:
      fbl[i] = 'p'
      count[c] -= 1
    else:
      fbl[i] = 'w'
  
  return "".join(fbl)

print(calc_feedback('slump', 'house'))

wwrpw


In [23]:
# Read feedback dict from file, or calculate and save
path = dir + 'fb_dict.pkl'
if exists(path):
  with open(path, 'rb') as file:
    fb_dict = pickle.load(file)
else:
  # Calculate feedback dictionary
  fb_dict = {}
  for true_word in solutions:
    d = {}
    for test_word in guesses:
      d[test_word] = calc_feedback(true_word, test_word)
    fb_dict[true_word] = d
  # Save fb_dict
  with open(path, 'wb') as file:
    pickle.dump(fb_dict, file)

In [24]:
# Calculate test_word entropy for list ot true words
def test_word_entropy(true_words, test_word):
  feedback_list = [fb_dict[true_word][test_word] for true_word in true_words]
  count = Counter(feedback_list)
  dist = list(count.values())
  return distribution_to_entropy(dist)


In [25]:
def test_word_entropies(true_words, test_words):
  d = dict()
  for test_word in test_words:
    d[test_word] = test_word_entropy(true_words, test_word)
  df = pd.DataFrame.from_dict(d, orient='index', columns=['Entropy'])
  return df


In [26]:
# Get dataframe of test word entropies
path = '/content/drive/MyDrive/ColabData/Wordle/entropies.pkl'
if exists(path):
  df = pd.read_pickle(path)
else:
  df = test_word_entropies(solutions, guesses)
  df.sort_values(by='Entropy', inplace=True, ascending=True)
  pd.DataFrame.to_pickle(df, path)

print(df.head)
print(list(df['Entropy'][df['Entropy']<1].index))

<bound method NDFrame.head of         Entropy
soare  5.290836
roate  5.294017
raise  5.298887
raile  5.311087
reast  5.311339
...         ...
yukky  8.971531
xylyl  8.984796
immix  9.124197
jujus  9.138487
qajaq  9.284960

[12972 rows x 1 columns]>
[]


In [27]:
class Generic_Node:
  """
  Generic search tree node
  """

  def __init__(self, parent, valid_words):
    self.parent = parent
    self.valid_words = valid_words
    self.value = None
    self.children = []

  def parent_branch(self):
    # Text representation of branch that resulted in this node
    raise Exception("Abstract class")

  def path(self):
    # Text representation of path from roor to this node
    path = []
    node = self
    while node:
      path.insert(0, node.parent_branch())
      node = node.parent
    return(str(path))

In [60]:
class Guessing_Node(Generic_Node):
  """
  Represents node in search tree where player has to decide on a guess word
  """
  
  def __init__(self, parent, valid_words, pattern, probability):
    """
    Initialize node with list of valid words
    """
    if len(valid_words) <1:
      raise Exception("No valid words.")
    super().__init__(parent, valid_words)
    self.pattern = pattern
    self.probability = probability
    self.best_guess = None

  def search(self):
    """
    Calculate value of expected steps
    """
#    print(self.path())
    valid_word_count = len(self.valid_words)

    if self.pattern == 'rrrrr':
      self.value = 0
    elif valid_word_count == 1:
      self.value = 1
    elif valid_word_count == 2:
      self.value = 1.5
    else:
      # If small number of valid words make sure to check all to go for direct hit
      if valid_word_count <= 5:
        direct_guesses = self.valid_words
      else:
        direct_guesses = []
      # Use entropy to find best guesses
      df_ent = test_word_entropies(self.valid_words, guesses)
      df_ent.sort_values(by='Entropy', inplace=True)
      entropy_guesses = list(df_ent['Entropy'][0:4].index)
      added_guesses = [g for g in entropy_guesses if g not in direct_guesses]
      best_guesses = direct_guesses + added_guesses

      # Use optimal value as stop condition
      optimal_child_value = (valid_word_count - 1) / valid_word_count

      for guess in best_guesses:
        child = Feedback_Node(self, self.valid_words, guess)
        self.children.append(child)
        child.search()
        if child.value == optimal_child_value:
          break
      child_values = [c.value for c in self.children]
      min_child_value = min(child_values)
      self.value = 1 + min_child_value
      best_guess = self.children[child_values.index(min_child_value)]

  def parent_branch(self):
    return '{} {} {:.2f}'.format(self.pattern, len(self.valid_words), 
                                 self.probability)
    
  def children_summary(self):
    data = {c.guess: c.value for c in self.children}
    df = pd.DataFrame.from_dict(data, orient='index', columns=['Value'])
    df.sort_values(by='Value', axis=0, ascending =True, inplace=True)
    return df

In [40]:
class Feedback_Node(Generic_Node):
  """
  Represents node in search tree where player entered a guess word and is about
  to receive feedback. This is a stochastic step.
  """
  def __init__(self, parent, valid_words, guess):
    """
    Initialize node with list of valid words
    """
    if len(valid_words) < 1:
      raise Exception("No valid words")
    super().__init__(parent, valid_words)
    self.guess = guess

  def search(self):
    """
    Calculate value of expected steps
    """
    total_valid = len(self.valid_words)
    feedbacks = {w: fb_dict[w][self.guess] for w in self.valid_words}
    patterns = {p: [w for w in self.valid_words if feedbacks[w] == p] 
                for p in set(feedbacks.values())}

    # Check that feedback reduced problem to avoid recursion issue
    if len(patterns) == 1:
      # Only one feedback, dead end, return max float
      self.value = sys.float_info.max
      return

    pattern_probs = {p: len(patterns[p]) / total_valid for p in patterns}
    for p in patterns:
      child = Guessing_Node(self, patterns[p], p, pattern_probs[p])
      self.children.append(child)
      child.search()
    child_values = [c.value for c in self.children]
    self.value = sum([c.probability * c.value for c in self.children])

  def parent_branch(self):
    return '{}'.format(self.guess)

  def children_summary(self):
    data = {c.pattern: (len(c.valid_words), c.probability, c.value) 
                        for c in self.children}
    df = pd.DataFrame.from_dict(data, orient='index', 
                                columns=['Words', 'Probability', 'Value'])
    df.sort_values(by='Words', axis=0, ascending =False, inplace=True)
    return df

In [48]:
# Calculate first guesses
path = '/content/drive/MyDrive/ColabData/Wordle/first_guesses.pkl'
if exists(path):
  first_guesses = pd.read_pickle(path)
else:
  root = Guessing_Node(None, solutions, '', 1)
  root.search()
  first_guesses = root.children_summary()
  pd.DataFrame.to_pickle(first_guesses, path)

print(first_guesses)


          Value
soare  2.464363
roate  2.468251


In [49]:
node = Feedback_Node(None, solutions, 'salet')
node.search()
print(node.children_summary())

       Words  Probability     Value
wwwww    221     0.095464  2.800905
wwwpw    165     0.071274  2.745455
wwwrw    107     0.046220  2.850467
wpwww    102     0.044060  2.588235
wwpww     99     0.042765  2.656566
...      ...          ...       ...
wprpp      1     0.000432  1.000000
prppw      1     0.000432  1.000000
prwwr      1     0.000432  1.000000
wrrrr      1     0.000432  1.000000
rrwwr      1     0.000432  1.000000

[148 rows x 3 columns]


In [50]:
node.children_summary()

,Words,Probability,Value
wwwww,221,0.095464,2.800905
wwwpw,165,0.071274,2.745455
wwwrw,107,0.046220,2.850467
wpwww,102,0.044060,2.588235
wwpww,99,0.042765,2.656566
...,...,...,...
wprpp,1,0.000432,1.000000
prppw,1,0.000432,1.000000
prwwr,1,0.000432,1.000000
wrrrr,1,0.000432,1.000000


In [51]:
def create_filter(test_word, feedback):
  """
  Takes a word and the corresponding Wordle feedback, and created a function
  that checks if a true word is consitent with this feedback
  word is a 5 character string, and feedback is a 5 character string with
  r = right, p = partial, w = wrong
  """
  def is_valid(true_word):
    return (feedback == fb_dict[true_word][test_word])

  return is_valid

In [53]:
feedback = [
            ('soare', 'wwpww')
            #,('girsh', 'wpwww')
            #,('focus', 'wrrwr')
]
valid = solutions
for i in feedback:
  f = create_filter(i[0], i[1])
  valid = list(filter(f, valid))
  print(len(valid))
  print(valid)

138
['naval', 'batty', 'maxim', 'audit', 'gamma', 'hatch', 'jaunt', 'badly', 'gaudy', 'vital', 'banal', 'tangy', 'panic', 'caulk', 'pupal', 'tacit', 'watch', 'natal', 'canny', 'album', 'awful', 'gawky', 'gaily', 'lilac', 'madam', 'wacky', 'aphid', 'patty', 'taunt', 'tibia', 'alpha', 'admit', 'dandy', 'valid', 'catch', 'fault', 'waltz', 'aptly', 'fanny', 'vault', 'daddy', 'aging', 'pagan', 'human', 'manga', 'final', 'nanny', 'handy', 'ninja', 'annul', 'aping', 'mania', 'inlay', 'lanky', 'china', 'caddy', 'amply', 'magic', 'macaw', 'candy', 'gayly', 'dally', 'tatty', 'laugh', 'aunty', 'cavil', 'cabin', 'fatty', 'fancy', 'villa', 'gaunt', 'mafia', 'vapid', 'tubal', 'align', 'datum', 'apply', 'bylaw', 'iliac', 'pizza', 'baggy', 'mammy', 'kappa', 'amity', 'faint', 'haunt', 'bawdy', 'latch', 'tawny', 'tidal', 'gamut', 'canal', 'paint', 'balmy', 'manic', 'cacti', 'tacky', 'patch', 'admin', 'tabby', 'tally', 'caput', 'habit', 'jazzy', 'fatal', 'titan', 'taffy', 'axial', 'daily', 'papal', 'avia

In [76]:
feedback = [
            ('soare', 'wwpwp')
            ,('canal', 'wpwww')
            ,('theta', 'wwpwp')
]
valid = solutions
for i in feedback:
  f = create_filter(i[0], i[1])
  valid = list(filter(f, valid))
  print(len(valid))
  print(valid)

root = Guessing_Node(None, valid, '', 1)
root.search()
print(root.children_summary())

68
['panel', 'delta', 'lapel', 'kebab', 'enema', 'cheat', 'abbey', 'pleat', 'ahead', 'fella', 'metal', 'alien', 'glean', 'label', 'eclat', 'equal', 'valet', 'laden', 'decay', 'clean', 'eaten', 'matey', 'hazel', 'alley', 'delay', 'ideal', 'haven', 'gavel', 'fecal', 'theta', 'amend', 'begat', 'penal', 'adept', 'medal', 'bleak', 'cheap', 'media', 'waxen', 'decal', 'pedal', 'bleat', 'cleat', 'facet', 'navel', 'agent', 'annex', 'bagel', 'abled', 'legal', 'began', 'taken', 'hyena', 'camel', 'angel', 'petal', 'fetal', 'cadet', 'apnea', 'tweak', 'wheat', 'gleam', 'cagey', 'pecan', 'plead', 'vegan', 'knead', 'mecca']
4
['abbey', 'theta', 'adept', 'media']
1
['abbey']
Empty DataFrame
Columns: [Value]
Index: []


In [75]:
print(fb_dict['abbey']['soare'])
print(fb_dict['abbey']['canal'])
print(fb_dict['abbey']['theta'])

wwpwp
wpwww
wwpwp
